In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import MESS
import numpy as np
import pandas as pd

from IPython.display import display

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
sp_dat = "/home/isaac/SGD_empirical/empirical_reunion_spiders/spider.dat"
sp_df = pd.read_csv(sp_dat, index_col=0)
sp_df[:5]

,abundance,pi
GL-01a,150,0.011126
GL-01b,107,0.023247
GL-01c,32,0.000252
GL-02,2,0.000000
GL-03,200,0.028499


In [7]:
simfile = "/home/isaac/Continuosity/MESS/analysis/full-parameter-estimation/SIMOUT.txt"

rgr = MESS.inference.Regressor(empirical_df=sp_df, simfile=simfile, algorithm="rf")
#rgr.set_targets(target_list="_lambda")
rgr.predict(select_features=False, quick=True, verbose=True)

Finding best model parameters.
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  20 | elapsed:   19.1s remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.6min finished


{'n_estimators': 600, 'min_samples_split': 5, 'bootstrap': True, 'max_depth': 110, 'min_samples_leaf': 4}
         alpha           J_m  ecological_strength         m  speciation_prob   _lambda
0  3100.373181  2.260748e+06             6.130783  0.007746          0.00062  0.823789


# Developing some kind of prediction uncertainty

## Predicting error with another ML model
This is a clever idea, not sure how kosher it is: https://qucit.com/a-simple-technique-to-estimate-prediction-intervals-for-any-regression-model_en/. Here's the gist: 

    # split the data in train a validation set
    X1, X2, y1, y2 = train_test_split(X, y, test_size=0.5)
    # base_model can be any regression model, a  
    # sklearn.ensemble.GradientBoostingRegressor for instance
    base_model.fit(X1, y1)
    base_prediction = base_model.predict(X2)
    # compute the prediction error vector on the validation set
    validation_error = (base_prediction - y2) ** 2
    error_model.fit(X2, validation_error)

    # compute the mean and standard deviation of the distribution
    mean = base_model.predict(X_test)
    st_dev = error_model.predict(X_test)

## skgarden.RandomForestQuantileRegressor
https://github.com/scikit-garden/scikit-garden
https://scikit-garden.github.io/examples/QuantileRegressionForests/

In [ ]:
### Use QuantileForests for quantile estimation
from skgarden import RandomForestQuantileRegressor
rfqr = RandomForestQuantileRegressor(random_state=0)
rfqr.fit(X, y)
y_mean = rfqr.predict(X)
y_median = rfqr.predict(X, 50)

In [ ]:
## sklearn GB has quantile regression built in (sort of)
This works okay for GB, I guess.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

alpha = 0.95

clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=1000, max_depth=5,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)

clf.fit(rgr.X, rgr.y)

# Make the prediction on the meshed x-axis
y_upper = clf.predict(rgr.empirical_sumstats)

clf.set_params(alpha=1.0 - alpha)
clf.fit(rgr.X, rgr.y)

# Make the prediction on the meshed x-axis
y_lower = clf.predict(rgr.empirical_sumstats)

clf.set_params(loss='ls')
clf.fit(rgr.X, rgr.y)

# Make the prediction on the meshed x-axis
y_pred = clf.predict(rgr.empirical_sumstats)

print(y_upper, y_lower, y_pred)

## forestci: Again, not sure if this is useful
    
    cd /home/isaac/Continuosity/MESS/util
    git clone https://github.com/scikit-learn-contrib/forest-confidence-interval.git
    cd forest-confidence-interval
    python setup.py install

In [ ]:
import forestci as fci
import sklearn.model_selection
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(rgr.X, rgr.y)
v = fci.random_forest_error(rgr.best_model, X_train, X_test)
v